## 1. 패키지

In [27]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt

from gensim.models import FastText

from sklearn.metrics.pairwise import cosine_similarity
import itertools

from tqdm import tqdm_notebook

## 2. 데이터

### 1) read csv

In [28]:
actor_df = pd.read_excel('./text_data/Actor_adjective_test.xlsx')
# webtoon_df = pd.read_csv('../text_data/Webtoon_Keys.csv', encoding='utf-8-sig')

### 2) data check

In [29]:
actor_df.head(2)

,Name,age,Sex,total_content,adjective
0,감우성,1970,남,기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규 박철 차광수 곽진영 김소이...,"성공하다, 예민하다, 평범하다, 후줄근하다, 아름답다, 강하다, 멋지다, 갑갑하다,..."
1,강경준,1983,남,초등학교 시절에는 야구 중학교 시절에는 농구선수였었다 농구를 소재로 한 드라마 마지...,"탄탄하다, 쿨하다, 다부지다, 좋아하다, 서글서글하다, 신기하다, 건강하다, 미끈하다"


In [30]:
webtoon_df.head(2)

,웹툰이름,등장인물,성별,나이,설명,keyword,5_keywords
0,폭풍의전학생,주인공,남,10대,최완서를 얼굴에 니킥을 작렬시켜 한방에 보내버리고 정해인을 박치기 한다음에 머리를 ...,"엄청나다, 상당하다, 태연하다, 뛰어나다, 가깝다, 굉장하다, 약하다, 진정하다, ...","당하다, 엄청나다, 굉장하다, 진정하다, 뛰어나다"
1,폭풍의전학생,최완서,남,10대,전 3반짱. 묘한 색깔의 피부와 부서진 콧잔등이 포인트. 전설의 레전드의 최대의 수...,"꼬이다, 불쌍하다, 많다, 멀쩡하다, 수많다, 평범하다, 좋아하다, 강하다, 평정하...","약하다, 강하다, 강하다, 멀쩡하다, 평범하다"


## 3. 배우 키워드 추출

### 1) 모델 로드

In [2]:
model = FastText.load('./model/fasttext.model')

### 2) Max Sum Similarity 함수 정의

In [14]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
        
    words_vals = [words[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])#자기 자신과의 유사도 제외.
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

### 3) 입력 데이터 정의

In [31]:
# 배우 내용과 형용사 가져오기
actor_content_list = actor_df['total_content'].to_list()
actor_adjective_list = []

In [32]:
for i in actor_df['adjective']:
    actor_adjective_list.append(i.split(', '))

### 4) 배우 content 전처리

- 문장 정리 : 특수문자, 숫자 제거

In [33]:
okt = Okt()
token_actor_content = []

for i in tqdm_notebook(actor_content_list):
    i = re.sub('[^가-힣 ]', '', i)
    token_actor_content.append(okt.morphs(i))

/tmp/ipykernel_6033/3164475459.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(actor_content_list):


  0%|          | 0/1363 [00:00<?, ?it/s]

- content 임베딩 : Doc Embedding

In [34]:
embedding_actor_content = []

for content in tqdm_notebook(token_actor_content):
    content_vec = None
    
    for word in content:
        word_vec = model.wv.get_vector(word)
    
        if content_vec is None:
            content_vec = word_vec
        else:
            content_vec = content_vec + word_vec
    
    if content_vec is not None:
        content_vec = content_vec / len(content)
        embedding_actor_content.append(content_vec)

/tmp/ipykernel_6033/375247513.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for content in tqdm_notebook(token_actor_content):


  0%|          | 0/1363 [00:00<?, ?it/s]

### 5) 키워드 추출

In [35]:
# 배우 키워드 추출
keywords = []

for i in tqdm_notebook(range(len(embedding_actor_content))):
    # 임베딩
    doc_embedding = embedding_actor_content[i].reshape(1, -1)
    candidate_embeddings = [model.wv.get_vector(word) for word in actor_adjective_list[i]]
    
    # keyword 추출
    if len(actor_adjective_list[i]) >= 10:
        keyword = max_sum_sim(doc_embedding, candidate_embeddings, actor_adjective_list[i], 5, 10)
        keywords.append(", ".join(keyword))
    elif (len(actor_adjective_list[i]) < 10) & (len(actor_adjective_list[i]) > 5):
        keyword = max_sum_sim(doc_embedding, candidate_embeddings, actor_adjective_list[i], 5, len(actor_adjective_list[i]))
        keywords.append(", ".join(keyword))
    else:
        keywords.append(", ".join(actor_adjective_list[i])) # 형용사의 개수가 10개 미만이면 그대로 키워드로 만들기

/tmp/ipykernel_6033/143565965.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(embedding_actor_content))):


  0%|          | 0/1363 [00:00<?, ?it/s]

- 키워드 확인

In [36]:
keywords

['성장하다, 뛰어나다, 성공하다, 따뜻하다, 특별하다',
 '좋아하다, 신기하다, 탄탄하다, 건강하다, 다부지다',
 '밉다, 좋아하다, 어렵다, 유일하다, 위대하다',
 '당연하다, 지독하다, 애틋하다, 무능하다, 가난하다',
 '유쾌하다, 유일하다, 따뜻하다, 굉장하다, 흔하다',
 '밉다, 무섭다, 힘없다, 젊다, 아름답다',
 '낮다, 나쁘다, 바쁘다, 자리다, 유일하다',
 '찌질하다',
 '좋아하다, 맵다',
 '짧다, 부드럽다, 차분하다, 휘하다, 비열하다',
 '나쁘다, 유명하다, 끊임없다, 영원하다, 엄청나다',
 '작다',
 '나쁘다, 신기하다, 힘드다, 뻔하다, 친절하다',
 '새롭다, 뛰어나다, 소중하다, 굉장하다, 수상하다',
 '까맣다, 잘생기다, 드물다, 굵다, 미워하다',
 '귀엽다, 차갑다, 당당하다, 강하다, 성장하다',
 '어렵다, 착하다, 신기하다, 굉장하다, 수상하다',
 '잘생기다, 무섭다, 가깝다, 중요하다, 꾸준하다',
 '정확하다, 독특하다, 강하다',
 '하얗다',
 '젊다, 수많다, 맛있다, 유창하다, 흔하다',
 '밉다, 귀엽다, 슬프다, 굉장하다, 수상하다',
 '찬란하다, 실패하다, 성공하다, 뛰어나다, 신선하다',
 '새롭다, 멋지다, 강하다, 변하다, 엄청나다',
 '뛰어나다, 수상하다, 독특하다, 강렬하다, 힘드다',
 '게으르다, 부족하다, 자세하다, 엄청나다, 선하다',
 '자상하다, 새롭다, 엄청나다, 가득하다, 완전하다',
 '정확하다, 수상하다',
 '날카롭다, 밉다, 좋아하다',
 '어렵다, 꼬이다, 망하다, 여전하다, 꾸준하다',
 '성장하다, 충실하다, 위험하다, 무사하다, 완전하다',
 '점잖다, 깔끔하다, 수상하다, 좋아하다',
 '유명하다, 푸근하다, 부패하다',
 '야하다, 멋지다, 진정하다, 유쾌하다, 꾸준하다',
 '빨갛다, 옅다, 힘들다, 당연하다, 굉장하다',
 '미치다, 힘드다',
 '성장하다',
 '힘들다, 유명하다, 끊임없다, 불가능하다, 굉장하다',
 '예쁘다, 낮

### 6) actor_df : 5_keywords columns 추가

In [37]:
actor_df['5_keywords'] = keywords
actor_df.head(2)

,Name,age,Sex,total_content,adjective,5_keywords
0,감우성,1970,남,기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규 박철 차광수 곽진영 김소이...,"성공하다, 예민하다, 평범하다, 후줄근하다, 아름답다, 강하다, 멋지다, 갑갑하다,...","성장하다, 뛰어나다, 성공하다, 따뜻하다, 특별하다"
1,강경준,1983,남,초등학교 시절에는 야구 중학교 시절에는 농구선수였었다 농구를 소재로 한 드라마 마지...,"탄탄하다, 쿨하다, 다부지다, 좋아하다, 서글서글하다, 신기하다, 건강하다, 미끈하다","좋아하다, 신기하다, 탄탄하다, 건강하다, 다부지다"


In [38]:
save_df = actor_df[['Name', 'age', 'Sex', '5_keywords']]
save_df.head()

,Name,age,Sex,5_keywords
0,감우성,1970,남,"성장하다, 뛰어나다, 성공하다, 따뜻하다, 특별하다"
1,강경준,1983,남,"좋아하다, 신기하다, 탄탄하다, 건강하다, 다부지다"
2,강기영,1983,남,"밉다, 좋아하다, 어렵다, 유일하다, 위대하다"
3,강남길,1958,남,"당연하다, 지독하다, 애틋하다, 무능하다, 가난하다"
4,강동원,1981,남,"유쾌하다, 유일하다, 따뜻하다, 굉장하다, 흔하다"


In [42]:
save_df['age'] = save_df['age'].apply(lambda x : 2022 - x)
save_df

/tmp/ipykernel_6033/2072863834.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  save_df['age'] = save_df['age'].apply(lambda x : 2022 - x)


,Name,age,Sex,5_keywords
0,감우성,52,남,"성장하다, 뛰어나다, 성공하다, 따뜻하다, 특별하다"
1,강경준,39,남,"좋아하다, 신기하다, 탄탄하다, 건강하다, 다부지다"
2,강기영,39,남,"밉다, 좋아하다, 어렵다, 유일하다, 위대하다"
3,강남길,64,남,"당연하다, 지독하다, 애틋하다, 무능하다, 가난하다"
4,강동원,41,남,"유쾌하다, 유일하다, 따뜻하다, 굉장하다, 흔하다"
...,...,...,...,...
1358,황정민,53,여,"수상하다, 발랄하다, 성공하다, 뛰어나다, 엄청나다"
1359,황정음,38,여,수상하다
1360,황채림,26,여,"싫어하다, 당연하다, 차분하다, 성숙하다, 흔하다"
1361,김동주,68,여,"화려하다, 뛰어나다, 수상하다, 나란하다, 철저하다"


In [43]:
save_df.to_excel('./text_data/Actor_keywords_test.xlsx', encoding='utf-8-sig', index=False)

---
## 4. 웹툰 키워드 추출

* 함수는 배우 키워드 추출과 같은 함수를 사용한다.

In [3]:
okt = Okt()

In [5]:
webtoon_df = pd.read_excel('./text_data/Webtoon_adjective_test.xlsx')
webtoon_df

,웹툰이름,등장인물,성별,나이,설명,keyword
0,폭풍의전학생,주인공,남,10대,최완서를 얼굴에 니킥을 작렬시켜 한방에 보내버리고 정해인을 박치기 한다음에 머리를 ...,"엄청나다, 상당하다, 태연하다, 뛰어나다, 가깝다, 굉장하다, 약하다, 진정하다, ..."
1,폭풍의전학생,최완서,남,10대,전 3반짱. 묘한 색깔의 피부와 부서진 콧잔등이 포인트. 전설의 레전드의 최대의 수...,"꼬이다, 불쌍하다, 많다, 멀쩡하다, 수많다, 평범하다, 좋아하다, 강하다, 평정하..."
2,폭풍의전학생,김대용,남,10대,1반의 짱을 맡고 있는 녀석이다. 처음엔 무슨 최종보스같은 포스를 내뿜었지만 나중엔...,"우세하다, 비슷하다, 강하다, 강하다, 단련하다, 강하다, 친하다, 상당하다, 나쁘..."
3,폭풍의전학생,정해인,남,10대,실눈 남자. 서울에서 태어나고 자란 서울 토박이지만 어릴적 본 조폭 영화에서 큰 감...,"비겁하다, 비슷하다, 험악하다, 나쁘다, 친하다, 강하다,튼튼하다, 빠르다, 살벌하..."
4,폭풍의전학생,황인규,남,10대,근육질의 남자. 6반의 짱을 맡고 있다. 대체 어떻게 이런 녀석이 다른 반의 짱들과...,"우락부락하다, 강하다, 순진하다, 새롭다, 반하다, 순진하다, 더럽다, 당하다, 친..."
...,...,...,...,...,...,...
522,언터처블,이모란,여,20대,"맨 왼쪽 사진에서는 왼쪽이 이모란, 오른쪽이 이라기. 이시아의 언니. 안경을 쓴 미...","굉장하다, 무섭다, 말르다, 단순하다, 뻔하다, 미안하다"
523,언터처블,이라기,남,20대,"이모란, 이시아의 동생. 20살 대학생. 원래는 모범생이어서 수수했으나 대학에 들어...","수수하다, 잘생기다, 침착하다, 친하다, 자상하다"
524,언터처블,최준혁,남,20대,뱀파이어 중에서도 특이한 뱀파이어로 사람의 생체에너지를 흡수하는데에 한계가 없기 때...,"특이하다, 가능하다, 뻔하다, 상관없다, 많다, 당연하다, 태연하다, 여전하다, 새..."
525,언터처블,모모코,여,30대,번외편에서만 등장한 일본인. 최준혁이 일본으로 넘어가서 편집장 일을 하게 되어서 만...,"귀엽다, 잘생기다, 굉장하다, 순수하다, 귀엽다, 당하다, 많다, 순수하다, 좋아하..."


### 1) 입력 데이터 정의

In [6]:
# 1. 키워드가 없는 인물 제거
webtoon_df[webtoon_df['keyword'].isna()]

,웹툰이름,등장인물,성별,나이,설명,keyword


In [19]:
webtoon_df = webtoon_df[~webtoon_df['keyword'].isna()]

In [20]:
# 2. 키워드가 10개 미만인 데이터 Null값으로 치환
webtoon_df['keyword'] = webtoon_df['keyword'].apply(lambda x : x if len(x.split()) >= 10 else np.nan)

/tmp/ipykernel_8088/2956144884.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  webtoon_df['keyword'] = webtoon_df['keyword'].apply(lambda x : x if len(x.split()) >= 10 else np.nan)


In [21]:
webtoon_df[webtoon_df['keyword'].isna()] # 키워드가 10개 미만인 데이터

,웹툰이름,등장인물,성별,나이,설명,keyword
3,폭풍의전학생,정해인,남,10대,실눈 남자. 서울에서 태어나고 자란 서울 토박이지만 어릴적 본 조폭 영화에서 큰 감...,NaN
4,폭풍의전학생,황인규,남,10대,근육질의 남자. 6반의 짱을 맡고 있다. 대체 어떻게 이런 녀석이 다른 반의 짱들과...,NaN
8,폭풍의전학생,이승재,남,10대,웹툰에 오면서 새로 등장한 인물. 백발 적안이다. 1학년이지만 복학생이라 주인공들과...,NaN
14,모태솔로수용소,공기영,여,20대,"보라색의 단발 머리이며, 둥근 안경을 쓴 여자. 218호에 수감되어 있다. 17화에...",NaN
16,모태솔로수용소,dj.8(팔Eight),남,20대,본명은 박봉팔. 벽창오의 파트너. 선글라스와 모자를 착용하며 사투리를 쓴다. 독고남...,NaN
...,...,...,...,...,...,...
520,언터처블,바류,남,20대,금발벽안에 살짝 키가 작은 남성 뱀파이어. 본명은 진우. 시아가 이시아와는 소꿉친구...,NaN
521,언터처블,윤시환,남,20대,바류와 오르니즈에서 활동하는 아이돌이다. 110화에서 바류의 언급에 의하면 리더라...,NaN
522,언터처블,이모란,여,20대,"맨 왼쪽 사진에서는 왼쪽이 이모란, 오른쪽이 이라기. 이시아의 언니. 안경을 쓴 미...",NaN
523,언터처블,이라기,남,20대,"이모란, 이시아의 동생. 20살 대학생. 원래는 모범생이어서 수수했으나 대학에 들어...",NaN


In [22]:
# 3. 키워드가 10개 미만인 데이터 제거
webtoon_df = webtoon_df[~webtoon_df['keyword'].isna()]

In [7]:
# 배우 내용과 형용사 가져오기
webtoon_content_list = webtoon_df['설명'].to_list()
webtoon_adjective_list = []

In [8]:
for i in webtoon_df['keyword']:
    webtoon_adjective_list.append(i.split(', '))

### 2) 웹툰 설명 전처리

- 숫자 제거, 특수문자 제거, 형태소 토큰화

In [10]:
token_webtoon_content = []

for i in tqdm_notebook(webtoon_content_list):
    i = re.sub('[^가-힣 ]', '', i)
    token_webtoon_content.append(okt.morphs(i))

/tmp/ipykernel_6033/4064461745.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(webtoon_content_list):


  0%|          | 0/527 [00:00<?, ?it/s]

- 설명 임베딩 : Doc Embedding

In [11]:
embedding_webtoon_content = []

for content in tqdm_notebook(token_webtoon_content):
    content_vec = None
    
    for word in content:
        word_vec = model.wv.get_vector(word)
    
        if content_vec is None:
            content_vec = word_vec
        else:
            content_vec = content_vec + word_vec
    
    if content_vec is not None:
        content_vec = content_vec / len(content)
        embedding_webtoon_content.append(content_vec)

/tmp/ipykernel_6033/1736014212.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for content in tqdm_notebook(token_webtoon_content):


  0%|          | 0/527 [00:00<?, ?it/s]

### 3) 키워드 추출

In [15]:
# 배우 키워드 추출
keywords = []

for i in tqdm_notebook(range(len(embedding_webtoon_content))):
    # 임베딩
    doc_embedding = embedding_webtoon_content[i].reshape(1, -1)
    candidate_embeddings = [model.wv.get_vector(word) for word in webtoon_adjective_list[i]]
    
    # keyword 추출
    if len(webtoon_adjective_list[i]) >= 10:
        keyword = max_sum_sim(doc_embedding, candidate_embeddings, webtoon_adjective_list[i], 5, 10)
        keywords.append(", ".join(keyword))
    elif (len(webtoon_adjective_list[i]) < 10) & (len(webtoon_adjective_list[i]) > 5):
        keyword = max_sum_sim(doc_embedding, candidate_embeddings, webtoon_adjective_list[i], 5, len(webtoon_adjective_list[i]))
        keywords.append(", ".join(keyword))
    else:
        keywords.append(", ".join(webtoon_adjective_list[i]))

/tmp/ipykernel_6033/3276524923.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(embedding_webtoon_content))):


  0%|          | 0/527 [00:00<?, ?it/s]

- 키워드 확인

In [16]:
keywords

['당하다, 엄청나다, 굉장하다, 진정하다, 뛰어나다',
 '약하다, 강하다, 강하다, 멀쩡하다, 평범하다',
 '나쁘다, 친하다, 비슷하다, 상당하다, 성장하다',
 '나쁘다, 빠르다, 친하다, 날카롭다, 비슷하다',
 '나쁘다, 더럽다, 친하다, 당하다, 새롭다',
 '예쁘다, 아쉽다, 좋아하다, 새롭다, 친하다',
 '엄청나다, 엄청나다, 강하다, 뻔하다, 멀쩡하다',
 '좋아하다, 비슷하다, 확실하다, 화려하다, 충분하다',
 '안되다, 나쁘다, 당하다, 속이다, 새롭다',
 '당하다, 필요하다, 새롭다, 경박하다, 엄청나다',
 '좋아하다, 어리다, 나쁘다, 도망가다, 속이다',
 '많다, 뒤늦다, 당하다, 딱하다, 수상하다',
 '노랗다, 잘생기다, 쓰리다, 당하다, 자랑스럽다',
 '무섭다, 당하다, 뻔뻔스럽다, 괴롭다, 성공하다',
 '둥글다, 안되다, 비슷하다, 상관없다, 속이다',
 '좋아하다, 부끄럽다, 당하다, 야하다, 여리다',
 '막는다, 재미있다, 어느새다, 당하다, 속이다',
 '싫다, 나쁘다, 어리다, 속이다, 치사하다',
 '무섭다, 나쁘다, 빡치다, 속이다, 치사하다',
 '귀엽다, 작다, 많다, 옳다, 당하다',
 '늙다, 많다, 당하다, 돌아오다, 갇히다',
 '고맙다, 나쁘다, 쓰리다, 속이다, 다양하다',
 '좋아하다, 더럽다, 기쁘다, 강하다, 새롭다',
 '좋아하다, 당하다, 느리다, 착하다, 친절하다',
 '어둡다, 많다, 나쁘다, 속이다, 엄청나다',
 '작다, 안되다, 나쁘다, 불쌍하다, 치사하다',
 '예쁘다, 나쁘다, 빡치다, 당하다, 주도하다',
 '어둡다, 무섭다, 나쁘다, 속이다, 강하다',
 '크다, 무섭다, 나쁘다, 당하다, 속이다',
 '밉다, 귀엽다, 귀찮다, 나쁘다, 당하다',
 '높다, 특이하다, 똑똑하다, 위대하다, 불완전하다',
 '날카롭다, 당하다, 화나다, 거만하다, 당돌하다',
 '검다, 어둡다, 이상하다, 강하다, 단순하다',
 '아쉽다, 능글맞다, 좋아하다, 세련되다, 속이다',

### 4) webtoon_df : 5_keywords columns 추가

In [17]:
webtoon_df['5_keywords'] = keywords

In [30]:
webtoon_df = webtoon_df.reset_index(drop=True)

In [18]:
webtoon_df

,웹툰이름,등장인물,성별,나이,설명,keyword,5_keywords
0,폭풍의전학생,주인공,남,10대,최완서를 얼굴에 니킥을 작렬시켜 한방에 보내버리고 정해인을 박치기 한다음에 머리를 ...,"엄청나다, 상당하다, 태연하다, 뛰어나다, 가깝다, 굉장하다, 약하다, 진정하다, ...","당하다, 엄청나다, 굉장하다, 진정하다, 뛰어나다"
1,폭풍의전학생,최완서,남,10대,전 3반짱. 묘한 색깔의 피부와 부서진 콧잔등이 포인트. 전설의 레전드의 최대의 수...,"꼬이다, 불쌍하다, 많다, 멀쩡하다, 수많다, 평범하다, 좋아하다, 강하다, 평정하...","약하다, 강하다, 강하다, 멀쩡하다, 평범하다"
2,폭풍의전학생,김대용,남,10대,1반의 짱을 맡고 있는 녀석이다. 처음엔 무슨 최종보스같은 포스를 내뿜었지만 나중엔...,"우세하다, 비슷하다, 강하다, 강하다, 단련하다, 강하다, 친하다, 상당하다, 나쁘...","나쁘다, 친하다, 비슷하다, 상당하다, 성장하다"
3,폭풍의전학생,정해인,남,10대,실눈 남자. 서울에서 태어나고 자란 서울 토박이지만 어릴적 본 조폭 영화에서 큰 감...,"비겁하다, 비슷하다, 험악하다, 나쁘다, 친하다, 강하다,튼튼하다, 빠르다, 살벌하...","나쁘다, 빠르다, 친하다, 날카롭다, 비슷하다"
4,폭풍의전학생,황인규,남,10대,근육질의 남자. 6반의 짱을 맡고 있다. 대체 어떻게 이런 녀석이 다른 반의 짱들과...,"우락부락하다, 강하다, 순진하다, 새롭다, 반하다, 순진하다, 더럽다, 당하다, 친...","나쁘다, 더럽다, 친하다, 당하다, 새롭다"
...,...,...,...,...,...,...,...
522,언터처블,이모란,여,20대,"맨 왼쪽 사진에서는 왼쪽이 이모란, 오른쪽이 이라기. 이시아의 언니. 안경을 쓴 미...","굉장하다, 무섭다, 말르다, 단순하다, 뻔하다, 미안하다","무섭다, 말르다, 미안하다, 굉장하다, 단순하다"
523,언터처블,이라기,남,20대,"이모란, 이시아의 동생. 20살 대학생. 원래는 모범생이어서 수수했으나 대학에 들어...","수수하다, 잘생기다, 침착하다, 친하다, 자상하다","수수하다, 잘생기다, 침착하다, 친하다, 자상하다"
524,언터처블,최준혁,남,20대,뱀파이어 중에서도 특이한 뱀파이어로 사람의 생체에너지를 흡수하는데에 한계가 없기 때...,"특이하다, 가능하다, 뻔하다, 상관없다, 많다, 당연하다, 태연하다, 여전하다, 새...","특이하다, 당연하다, 당연하다, 새롭다, 여전하다"
525,언터처블,모모코,여,30대,번외편에서만 등장한 일본인. 최준혁이 일본으로 넘어가서 편집장 일을 하게 되어서 만...,"귀엽다, 잘생기다, 굉장하다, 순수하다, 귀엽다, 당하다, 많다, 순수하다, 좋아하...","귀엽다, 작다, 많다, 좋아하다, 당하다"


In [19]:
save_df = webtoon_df.rename({'웹툰이름' : 'Title', '등장인물' : 'Name', '나이' : 'Age', '성별':'Sex'}, axis=1)
save_df.head()

,Title,Name,Sex,Age,설명,keyword,5_keywords
0,폭풍의전학생,주인공,남,10대,최완서를 얼굴에 니킥을 작렬시켜 한방에 보내버리고 정해인을 박치기 한다음에 머리를 ...,"엄청나다, 상당하다, 태연하다, 뛰어나다, 가깝다, 굉장하다, 약하다, 진정하다, ...","당하다, 엄청나다, 굉장하다, 진정하다, 뛰어나다"
1,폭풍의전학생,최완서,남,10대,전 3반짱. 묘한 색깔의 피부와 부서진 콧잔등이 포인트. 전설의 레전드의 최대의 수...,"꼬이다, 불쌍하다, 많다, 멀쩡하다, 수많다, 평범하다, 좋아하다, 강하다, 평정하...","약하다, 강하다, 강하다, 멀쩡하다, 평범하다"
2,폭풍의전학생,김대용,남,10대,1반의 짱을 맡고 있는 녀석이다. 처음엔 무슨 최종보스같은 포스를 내뿜었지만 나중엔...,"우세하다, 비슷하다, 강하다, 강하다, 단련하다, 강하다, 친하다, 상당하다, 나쁘...","나쁘다, 친하다, 비슷하다, 상당하다, 성장하다"
3,폭풍의전학생,정해인,남,10대,실눈 남자. 서울에서 태어나고 자란 서울 토박이지만 어릴적 본 조폭 영화에서 큰 감...,"비겁하다, 비슷하다, 험악하다, 나쁘다, 친하다, 강하다,튼튼하다, 빠르다, 살벌하...","나쁘다, 빠르다, 친하다, 날카롭다, 비슷하다"
4,폭풍의전학생,황인규,남,10대,근육질의 남자. 6반의 짱을 맡고 있다. 대체 어떻게 이런 녀석이 다른 반의 짱들과...,"우락부락하다, 강하다, 순진하다, 새롭다, 반하다, 순진하다, 더럽다, 당하다, 친...","나쁘다, 더럽다, 친하다, 당하다, 새롭다"


In [20]:
save_df['Age'] = save_df.Age.str.replace('[^0-9]', '')
save_df

/tmp/ipykernel_6033/2158925986.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  save_df['Age'] = save_df.Age.str.replace('[^0-9]', '')


,Title,Name,Sex,Age,설명,keyword,5_keywords
0,폭풍의전학생,주인공,남,10,최완서를 얼굴에 니킥을 작렬시켜 한방에 보내버리고 정해인을 박치기 한다음에 머리를 ...,"엄청나다, 상당하다, 태연하다, 뛰어나다, 가깝다, 굉장하다, 약하다, 진정하다, ...","당하다, 엄청나다, 굉장하다, 진정하다, 뛰어나다"
1,폭풍의전학생,최완서,남,10,전 3반짱. 묘한 색깔의 피부와 부서진 콧잔등이 포인트. 전설의 레전드의 최대의 수...,"꼬이다, 불쌍하다, 많다, 멀쩡하다, 수많다, 평범하다, 좋아하다, 강하다, 평정하...","약하다, 강하다, 강하다, 멀쩡하다, 평범하다"
2,폭풍의전학생,김대용,남,10,1반의 짱을 맡고 있는 녀석이다. 처음엔 무슨 최종보스같은 포스를 내뿜었지만 나중엔...,"우세하다, 비슷하다, 강하다, 강하다, 단련하다, 강하다, 친하다, 상당하다, 나쁘...","나쁘다, 친하다, 비슷하다, 상당하다, 성장하다"
3,폭풍의전학생,정해인,남,10,실눈 남자. 서울에서 태어나고 자란 서울 토박이지만 어릴적 본 조폭 영화에서 큰 감...,"비겁하다, 비슷하다, 험악하다, 나쁘다, 친하다, 강하다,튼튼하다, 빠르다, 살벌하...","나쁘다, 빠르다, 친하다, 날카롭다, 비슷하다"
4,폭풍의전학생,황인규,남,10,근육질의 남자. 6반의 짱을 맡고 있다. 대체 어떻게 이런 녀석이 다른 반의 짱들과...,"우락부락하다, 강하다, 순진하다, 새롭다, 반하다, 순진하다, 더럽다, 당하다, 친...","나쁘다, 더럽다, 친하다, 당하다, 새롭다"
...,...,...,...,...,...,...,...
522,언터처블,이모란,여,20,"맨 왼쪽 사진에서는 왼쪽이 이모란, 오른쪽이 이라기. 이시아의 언니. 안경을 쓴 미...","굉장하다, 무섭다, 말르다, 단순하다, 뻔하다, 미안하다","무섭다, 말르다, 미안하다, 굉장하다, 단순하다"
523,언터처블,이라기,남,20,"이모란, 이시아의 동생. 20살 대학생. 원래는 모범생이어서 수수했으나 대학에 들어...","수수하다, 잘생기다, 침착하다, 친하다, 자상하다","수수하다, 잘생기다, 침착하다, 친하다, 자상하다"
524,언터처블,최준혁,남,20,뱀파이어 중에서도 특이한 뱀파이어로 사람의 생체에너지를 흡수하는데에 한계가 없기 때...,"특이하다, 가능하다, 뻔하다, 상관없다, 많다, 당연하다, 태연하다, 여전하다, 새...","특이하다, 당연하다, 당연하다, 새롭다, 여전하다"
525,언터처블,모모코,여,30,번외편에서만 등장한 일본인. 최준혁이 일본으로 넘어가서 편집장 일을 하게 되어서 만...,"귀엽다, 잘생기다, 굉장하다, 순수하다, 귀엽다, 당하다, 많다, 순수하다, 좋아하...","귀엽다, 작다, 많다, 좋아하다, 당하다"


In [23]:
save_df = save_df.fillna(0)

In [25]:
save_df = save_df[['Title', 'Name', 'Sex', 'Age', '5_keywords']]
save_df

,Title,Name,Sex,Age,5_keywords
0,폭풍의전학생,주인공,남,10,"당하다, 엄청나다, 굉장하다, 진정하다, 뛰어나다"
1,폭풍의전학생,최완서,남,10,"약하다, 강하다, 강하다, 멀쩡하다, 평범하다"
2,폭풍의전학생,김대용,남,10,"나쁘다, 친하다, 비슷하다, 상당하다, 성장하다"
3,폭풍의전학생,정해인,남,10,"나쁘다, 빠르다, 친하다, 날카롭다, 비슷하다"
4,폭풍의전학생,황인규,남,10,"나쁘다, 더럽다, 친하다, 당하다, 새롭다"
...,...,...,...,...,...
522,언터처블,이모란,여,20,"무섭다, 말르다, 미안하다, 굉장하다, 단순하다"
523,언터처블,이라기,남,20,"수수하다, 잘생기다, 침착하다, 친하다, 자상하다"
524,언터처블,최준혁,남,20,"특이하다, 당연하다, 당연하다, 새롭다, 여전하다"
525,언터처블,모모코,여,30,"귀엽다, 작다, 많다, 좋아하다, 당하다"


In [26]:
save_df.to_excel('./text_data/Webtoon_keywords_test.xlsx', encoding='utf-8-sig', index=False)

- 키워드가 10개 미만인 데이터를 모두 제거하였기 때문에 데이터가 275개로 줄음
    - 해결하고 싶다면 키워드를 채우거나 키워드 추출 개수를 줄이는 방법을 채택할 수 있다.